In [1]:
from datetime import datetime as dt
from datetime import timedelta
import logging
import importlib
import urllib
import unicodecsv as csv
from lxml import html
import requests
import pandas as pd
import numpy as np
import json
import sys
from requests.auth import HTTPProxyAuth
import time
import os

In [2]:
#Script is written right now for before cleaned, single region. Must adapt for cleaned and multi-region.
#This will include changing "price" to "rent"
#QUESTION: Unclear what is actually being counted in 'score' variable

cleandf = pd.read_csv('/Users/anniedbr/Desktop/CSV/data-modesto20170322-131051.csv',header=0)
#determine rent per square foot
cleandf['rent_sqft'] = cleandf['price'] / cleandf['sqft']

In [3]:
#List as 'true' if the PID, body_text, or the title are duplicates
cleandf['body_text_duplicate']=cleandf.duplicated('body_text')
cleandf['title_duplicate']=cleandf.duplicated('title')
cleandf['PID_duplicate']=cleandf.duplicated('pid')

In [4]:
#Give a score to each row based on the number of fields that are filled out. This will allow us to keep the most information
#for duplicate rows
cleandf['score']=cleandf.astype(bool).sum(axis=1)
cleandf = cleandf.head().sort_values(by='score',ascending=False)
cleandf.head()

,pid,dt,url,title,price,neighb,sqft,lat,lng,accuracy,...,private_room,bath_known,private_bath,parking_known,onsite_parking,rent_sqft,body_text_duplicate,title_duplicate,PID_duplicate,score
3,6017847804,2017-03-22 12:29,http://modesto.craigslist.org/roo/6017847804.html,Beautiful single villa rental!,600.0,MODESTO,460,37.671700,-120.948200,22,...,True,True,True,True,False,1.304348,True,True,False,21
4,6017279149,2017-03-22 12:29,http://modesto.craigslist.org/roo/6017279149.html,Beautiful single villa rental!,600.0,MODESTO,460,37.677163,-120.949345,7,...,True,True,False,True,False,1.304348,True,True,False,20
1,6023614883,2017-03-22 12:30,http://modesto.craigslist.org/roo/6023614883.html,Beautiful single villa rental!,NaN,MODESTO,0,37.674600,-121.011300,22,...,True,True,False,True,False,NaN,True,True,False,19
2,6015724024,2017-03-22 12:29,http://modesto.craigslist.org/roo/6015724024.html,Beautiful single villa rental!,NaN,MODESTO,0,37.679473,-120.944366,7,...,True,True,False,True,False,NaN,True,True,False,19
0,6023610939,2017-03-22 12:30,http://modesto.craigslist.org/roo/6023610939.html,Beautiful single villa rental!,NaN,MODESTO,0,37.674600,-121.011300,22,...,True,True,False,True,False,NaN,False,False,False,17


In [5]:
#Keep rows with highest score. Drop rows with duplicate PIDs, Duplicate Titles, Duplicate Posting Bodies
cleandf_unique = cleandf.drop_duplicates('body_text')
cleandf_unique.head()

,pid,dt,url,title,price,neighb,sqft,lat,lng,accuracy,...,private_room,bath_known,private_bath,parking_known,onsite_parking,rent_sqft,body_text_duplicate,title_duplicate,PID_duplicate,score
3,6017847804,2017-03-22 12:29,http://modesto.craigslist.org/roo/6017847804.html,Beautiful single villa rental!,600.0,MODESTO,460,37.6717,-120.9482,22,...,True,True,True,True,False,1.304348,True,True,False,21


In [6]:
cleandf_unique = cleandf.drop_duplicates('pid')
cleandf_unique = cleandf.drop_duplicates('title')
cleandf_unique.head()

,pid,dt,url,title,price,neighb,sqft,lat,lng,accuracy,...,private_room,bath_known,private_bath,parking_known,onsite_parking,rent_sqft,body_text_duplicate,title_duplicate,PID_duplicate,score
3,6017847804,2017-03-22 12:29,http://modesto.craigslist.org/roo/6017847804.html,Beautiful single villa rental!,600.0,MODESTO,460,37.6717,-120.9482,22,...,True,True,True,True,False,1.304348,True,True,False,21


In [ ]:
# calculate the ratios of unique to duplicate listings for each region
listings_ratios = pd.DataFrame()

# number of total listings for each region
listings_ratios['cleandf'] = cleandf['region'].value_counts()

# number of duplicate listings for each region (ie, listings that share a pid with at least one other listing)
listings_ratios['duplicate_listings'] = duplicate_listings['region'].value_counts()

# number of unique listings for the region (ie, none share a pid with another listing)
listings_ratios['unique_listings'] = unique_listings['region'].value_counts()

# percent of this region's listings that are duplicates
listings_ratios['duplicate_ratio'] = listings_ratios['duplicate_listings'] / listings_ratios['all_listings']

# percent of this region's listings that are unique (ie, not duplicates)
listings_ratios['unique_ratio'] = listings_ratios['unique_listings'] / listings_ratios['all_listings']

listings_ratios.head()